<a href="https://colab.research.google.com/github/SantiagoSF1/SwarmUI/blob/main/swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🐴 **Instalar entorno**
</details><img src='https://i.pinimg.com/originals/c1/bc/3d/c1bc3d6ba8b7a7c9ff037660e2e4f2c2.gif' width='800px'/>
<details>
  <summary><font color=gray>Últimos cambios</summary>

```diff
> ☢️ Importante: ejecute la opción indicada antes que nada.
> ☢️ Importante: tenga disponible la información necesaria.
> ☢️ Importante: no es necesario reiniciar el sistema.
> 🟢 SageMaker: mantendrá la instalación completada.
```
</details>

In [ ]:
Webui = 'SwarmUI' # @param ["A1111", "Forge", "ReForge", "Forge-Classic", "Forge-Neo", "ComfyUI", "SwarmUI"]
Civitai___Key = '65ebb54e49d7eaca8e20e03ede229a57' # @param { type: "string", placeholder: "Civitai" }
HF_Read_Token = '' # @param { type: "string", placeholder: "Huggingface" }
Mount__GDrive = 'No' # @param ["Yes", "No"]

mount = Mount__GDrive

if mount == 'Yes':
    from google.colab import drive
    drive.mount('/content/drive')

!curl -sLo /content/setup.py https://github.com/gutris1/segsmaker/raw/main/script/KC/setup.py
%run /content/setup.py --webui="$Webui" --civitai_key="$Civitai___Key" --hf_read_token="$HF_Read_Token"

if mount == 'Yes':
    from pathlib import Path

    d = Path('/content/drive/MyDrive/Segsmaker')

    for n, p in {'checkpoint': CKPT, 'lora': LORA, 'vae': VAE, 'embeddings': Embeddings}.items():
        f = d / n
        f.mkdir(parents=True, exist_ok=True)
        s = p / f'drive-{n}'
        s.symlink_to(f, target_is_directory=True)

    !rm -rf $WebUI_Output
    o = d / {'ComfyUI': 'comfyui-output', 'SwarmUI': 'swarmui-output'}.get(Webui, 'output')
    o.mkdir(parents=True, exist_ok=True)
    WebUI_Output.symlink_to(o, target_is_directory=True)

    if Webui not in {'ComfyUI', 'SwarmUI'}:
        wc = WebUI / 'cache'
        !rm -rf $wc
        c = d / 'cache'
        c.mkdir(parents=True, exist_ok=True)
        wc.symlink_to(c, target_is_directory=True)

In [ ]:
#@markdown #🤖  <b><font color='grey'>ComfyUI</font></b> {"display-mode":"form"}

import os

# Crear carpetas necesarias
os.makedirs("/content/SwarmUI/dlbackend", exist_ok=True)
os.makedirs("/content/SwarmUI/Output/local/examp", exist_ok=True)

# Clonar repos SwarmUI y Data
%cd -q /content/SwarmUI/dlbackend
!git clone https://github.com/comfyanonymous/ComfyUI.git

%cd -q /content/SwarmUI
!git clone https://github.com/SantiagoSF1/Data.git

# Descargar imagen específica dentro de la carpeta examp
%cd -q /content/SwarmUI/Output/local/examp
!wget -q https://huggingface.co/SantiagoSF1/Pony-Lora/resolve/main/Images/105015.png


#Fin

## 🦜 **Instalar modelos**
</details><img src='https://i.pinimg.com/originals/7f/ae/cb/7faecb6cee8f08f09b48d0547f870495.gif' width='800px'/>
<details>
  <summary><font color=gray>Últimos cambios</summary>

```diff
+ Descarga Modelo Pony Versión 6
+ Descarga Imágenes Pony Versión 6
```
</details>

In [ ]:
#@markdown # 🌀 <b><font color='grey'>Checkpoint</font></b> {"display-mode":"form"}
Modelo = "Margarita" #@param ["Margarita", "IWAI-Illustrious", "Blender"]

%cd -q $CKPT

if Modelo == "Margarita":
    %download https://huggingface.co/SantiagoSF1/checkpoint/resolve/main/Margarita.safetensors
elif Modelo == "IWAI-Illustrious":
    %download https://huggingface.co/SantiagoSF1/checkpoint/resolve/main/IWAI-Illustrious.safetensors
elif Modelo == "Blender":
    %download https://huggingface.co/SantiagoSF1/checkpoint/resolve/main/Blender.safetensors

In [ ]:
#@markdown # 🦜 <b><font color='grey'>LoRas</font></b> {"display-mode":"form"}

from huggingface_hub import HfApi
api = HfApi()
%cd -q $LORA

# Obtener lista de archivos
files = api.list_repo_files(
    repo_id="SantiagoSF1/illus-lora",
    repo_type="model")

# Descargar cada archivo (sin filtrar)
for file in files:
    url = f"https://huggingface.co/SantiagoSF1/illus-lora/resolve/main/{file}"
    get_ipython().run_line_magic("download", url)

In [ ]:
#@markdown # 🦜 <b><font color='grey'>LoRas Personalizados</font></b> {"display-mode":"form"}
custom_link = ""   #@param {type:"string"}
custom_name = "Lora"   #@param {type:"string"}

%cd -q $LORA

# Descargar archivo personalizado con nombre + .safetensors automático
if custom_link.strip() and custom_name.strip():
    filename = f"{custom_name}.safetensors"
    get_ipython().run_line_magic(
        "download",
        f"{custom_link} {filename}")

In [ ]:
#@markdown # 🗿 <b><font color='grey'>Controlnet</font></b> {"display-mode":"form"}

%cd -q /content/SwarmUI/Models/controlnet
%download https://huggingface.co/SantiagoSF1/controlnet/resolve/main/net-union.safetensors


## 🎮 **Iniciar entorno**
</details><img src='https://i.pinimg.com/originals/01/38/11/013811d63121a093c32714cbbeeea0d0.gif' width='800px'/>
<details>
  <summary><font color=gray>Últimos cambios</summary>

-  **A1111** = `--xformers`
- **Forge** = `--disable-xformers --opt-sdp-attention --cuda-stream`
- **ReForge** = `--xformers --cuda-stream`
- **Forge-Classic** = `--xformers --cuda-stream --persistent-patches`
- **Forge-Neo** = `--xformers --cuda-stream`
- **ComfyUI** = `--dont-print-server --use-pytorch-cross-attention`
- **SwarmUI** = `--launch_mode none`

In [ ]:
#@markdown # 🎮 <b><font color="#FFB3B3">I</font><font color="#FFD1A6">n</font><font color="#FFF2A6">i</font><font color="#C6F6C6">c</font><font color="#C6D8FF">i</font><font color="#D8C6FF">a</font><font color="#F2C6FF">r</font></b> {"display-mode":"form"}
#@markdown Prepara y ejecuta el entorno **SwarmUl**, configurando los módulos necesarios y dejando todo listo para trabajar.

%cd -q $WebUI
%run segsmaker.py --launch_mode none

## 📳 **Ajustes**
</details><img src='https://i.pinimg.com/originals/ac/39/58/ac3958e3e82f09c3d19facba53badd93.gif' width='800px'/>
<details>
  <summary><font color=gray>Últimos cambios</summary>

```diff
+ Crea un link para ejecutar
+ Ejecuta Stable diffusion
```
</details>

In [ ]:
#@markdown # 📬 <b><font color='grey'>Subir imagen</font></b> {"display-mode":"form"}

from google.colab import files
import os

# Ruta destino
destino = "/content/SwarmUI/Output/local/examp/"

# Subir imagen
uploaded = files.upload()

# Mover imagen a la carpeta destino
for nombre in uploaded.keys():
    os.rename(nombre, os.path.join(destino, nombre))

print("✅ Imagen subida correctamente a:", destino)

In [ ]:
''' SD Extensions / ComfyUI Custom Nodes '''
%cd -q $Extensions
!git clone

''' VAE '''
%cd -q $VAE
%download

''' Embeddings '''
%cd -q $Embeddings
%download

''' Upscalers '''
%cd -q $Upscalers
%download

''' FLUX Unet '''
%cd -q $UNET
%download

''' FLUX Clip '''
%cd -q $CLIP
%download

In [ ]:
''' Controlnet '''
%run $Controlnet_Widget

In [ ]:
%%zipping
name    = SwarmUl
inputs  = $WebUI_Output
outputs = $HOME